In [ ]:
# @title Setup
!pip install sodapy
!pip install unidecode
import pandas as pd
import requests
from sodapy import Socrata
from google.colab import drive
import os
import time
from IPython.display import display, Markdown
from google.colab import drive
from unidecode import unidecode
import json
import folium
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Carga de datos y dfs
evals = pd.read_csv('/content/drive/My Drive/DANE/CSVs listos/evals/evals.csv')
evals['Código Departamento'] = evals['Código Departamento'].astype(str).str.zfill(2)

url = 'https://gist.githubusercontent.com/john-guerra/43c7656821069d00dcbc/raw/3aadedf47badbdac823b00dbe259f6bc6d9e1899/colombia.geo.json'
colombia_geojson = requests.get(url).json()

evalsSembA = evals[['Año', 'Código Departamento', 'Área Sembrada (ha)']]
evalsSembA = evalsSembA.groupby(['Año', 'Código Departamento'])['Área Sembrada (ha)'].sum().reset_index()
evalsSemb = evalsSembA.groupby(['Código Departamento'])['Área Sembrada (ha)'].mean().reset_index()

evalsCosecA = evals[['Año', 'Código Departamento', 'Área Cosechada (ha)']]
evalsCosecA = evalsCosecA.groupby(['Año', 'Código Departamento'])['Área Cosechada (ha)'].sum().reset_index()
evalsCosec = evalsCosecA.groupby(['Código Departamento'])['Área Cosechada (ha)'].mean().reset_index()

evalsProdA = evals[['Año', 'Código Departamento', 'Producción (ton/ha)']]
evalsProdA = evalsProdA.groupby(['Año', 'Código Departamento'])['Producción (ton/ha)'].sum().reset_index()
evalsProd = evalsProdA.groupby(['Código Departamento'])['Producción (ton/ha)'].mean().reset_index()

evalsRendA = evals[['Año', 'Código Departamento', 'Rendimiento (ton_ha)']]
evalsRendA = evalsRendA.groupby(['Año', 'Código Departamento'])['Rendimiento (ton_ha)'].sum().reset_index()
evalsRend = evalsRendA.groupby(['Código Departamento'])['Rendimiento (ton_ha)'].mean().reset_index()

evalsIndCosec = evals[['Año', 'Código Departamento', 'Área Sembrada (ha)', 'Área Cosechada (ha)']].copy()

evalsIndCosec['Índice de Cosecha'] = evalsIndCosec.apply(
    lambda row: row['Área Cosechada (ha)'] / row['Área Sembrada (ha)'] if row['Área Sembrada (ha)'] > 0 else np.nan, axis=1
)

evalsIndCosecA = evalsIndCosec[['Año', 'Código Departamento', 'Índice de Cosecha']]
evalsIndCosecA = evalsIndCosecA.groupby(['Año', 'Código Departamento'])['Índice de Cosecha'].mean().reset_index()

evalsIndCosec = evalsIndCosecA.groupby(['Código Departamento'])['Índice de Cosecha'].mean().reset_index()

## Modelo

### Preparación de datos

In [ ]:
def ajustarCoordenadas(lat, lon):
  latAjustada = lat
  lonAjustada = lon
  return latAjustada, lonAjustada

In [ ]:
tempAntioquia = pd.read_csv('/content/drive/My Drive/DANE/CSVs listos/temp/tempANTIOQUIA.csv')
tempAntioquia['fechaobservacion'] = pd.to_datetime(tempAntioquia['fechaobservacion'])
tempAntioquia['Año'] = tempAntioquia['fechaobservacion'].dt.year
tempAntioquia['Mes'] = tempAntioquia['fechaobservacion'].dt.month

tempAntioquiaG = tempAntioquia.groupby(['codigoestacion', 'latitud', 'longitud', 'departamento', 'Mes'])['valorobservado'].mean().reset_index()
tempAntioquiaG

In [ ]:
velAntioquia = pd.read_csv('/content/drive/My Drive/DANE/CSVs listos/vel/velANTIOQUIA.csv')
velAntioquia['fechaobservacion'] = pd.to_datetime(velAntioquia['fechaobservacion'])
velAntioquia['Año'] = velAntioquia['fechaobservacion'].dt.year
velAntioquia['Mes'] = velAntioquia['fechaobservacion'].dt.month

velAntioquiaG = velAntioquia.groupby(['codigoestacion', 'latitud', 'longitud', 'departamento', 'Año', 'Mes'])['valorobservado'].mean().reset_index()
velAntioquiaG

In [ ]:
# @title Diccionarios
departamentos = [
'AMAZONAS',
'ANTIOQUIA',
'ARAUCA',
'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA',
'ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
'ATLANTICO',
'ATLÁNTICO',
'BOGOTA',
'BOGOTA D.C.',
'BOGOTÁ',
'BOLIVAR',
'BOLÍVAR',
'BOYACÁ',
'CALDAS',
'CAQUETA',
'CAQUETÁ',
'CASANARE',
'CAUCA',
'CESAR',
'CHOCO',
'CHOCÓ',
'CORDOBA',
'CUNDINAMARCA',
'CÓRDOBA',
'GUAINÍA',
'GUAVIARE',
'HUILA',
'LA GUAJIRA',
'MAGDALENA',
'META',
'NARINO',
'NARIÑO',
'NORTE DE SANTANDER',
'PUTUMAYO',
'QUINDÍO',
'RISARALDA',
'SAN ANDRÉS PROVIDENCIA',
'SANTANDER',
'SUCRE',
'TOLIMA',
'VALLE DEL CAUCA',
'VICHADA'
]

dictDeps = {
  'Antioquia': '05',
  'Atlantico': '08',
  'Bolivar': '13',
  'Bogota': '11',
  'Bogota D.C.':'11',
  'Boyaca': '15',
  'Caldas': '17',
  'Caqueta': '18',
  'Cauca': '19',
  'Cesar': '20',
  'Cordoba': '23',
  'Cundinamarca': '25',
  'Choco': '27',
  'Huila': '41',
  'La Guajira': '44',
  'Magdalena': '47',
  'Meta': '50',
  'Nariño': '52',
  'Narino': '52',
  'Norte De Santander': '54',
  'Quindio': '63',
  'Risaralda': '66',
  'Santander': '68',
  'Sucre': '70',
  'Tolima': '73',
  'Valle Del Cauca': '76',
  'Arauca': '81',
  'Casanare': '85',
  'Putumayo': '86',
  'San Andres Y Providencia': '88',
  'San Andres Providencia': '88',
  'Archipielago De San Andres Providencia Y Santa Catalina': '88',
  'Archipielago De San Andres, Providencia Y Santa Catalina': '88',
  'Amazonas': '91',
  'Guainia': '94',
  'Guaviare': '95',
  'Vaupes': '97',
  'Vichada': '99'
}

In [ ]:
dfTemp = pd.DataFrame()
for departamento in departamentos:
  df = pd.read_csv(f'/content/drive/My Drive/DANE/CSVs listos/temp/temp{departamento}.csv')
  df['fechaobservacion'] = pd.to_datetime(df['fechaobservacion'])
  df['Año'] = df['fechaobservacion'].dt.year

  df = df.groupby(['departamento', 'Año'])['valorobservado'].mean().reset_index()

  dfTemp = pd.concat([dfTemp, df])
dfTemp

In [ ]:
dfVel = pd.DataFrame()
for departamento in departamentos:
  df = pd.read_csv(f'/content/drive/My Drive/DANE/CSVs listos/vel/vel{departamento}.csv')
  df['fechaobservacion'] = pd.to_datetime(df['fechaobservacion'])
  df['Año'] = df['fechaobservacion'].dt.year

  df = df.groupby(['departamento', 'Año'])['valorobservado'].mean().reset_index()

  dfVel = pd.concat([dfVel, df])
dfVel

### Entrenamiento del Modelo

In [ ]:
# @title Carga de datos de Drive
dfTemp = pd.read_csv('/content/drive/My Drive/DANE/CSVs listos/temp/tempResumen.csv')
dfVel = pd.read_csv('/content/drive/My Drive/DANE/CSVs listos/vel/velResumen.csv')

# dfTemp['departamento'] = dfTemp['departamento'].apply(unidecode)
# dfTemp['departamento'] = dfTemp['departamento'].str.title()
# dfTemp['CodDep'] = dfTemp['departamento'].map(dictDeps)

# dfVel['departamento'] = dfVel['departamento'].apply(unidecode)
# dfVel['departamento'] = dfVel['departamento'].str.title()
# dfVel['CodDep'] = dfVel['departamento'].map(dictDeps)

# dfTemp['CodDep'] = dfTemp['CodDep'].astype(str).str.zfill(2)
# dfVel['CodDep'] = dfVel['CodDep'].astype(str).str.zfill(2)

In [ ]:
# @title DataFrame de entrenamiento
dfModelo = pd.merge(dfTemp[['CodDep', 'Año', 'valorobservado']], dfVel[['CodDep', 'Año', 'valorobservado']], on=['CodDep', 'Año'], suffixes=('_temp', '_vel'))
dfModelo['CodDep'] = dfModelo['CodDep'].astype(str).str.zfill(2)
dfModelo = pd.merge(dfModelo, evalsRendA, left_on=['CodDep', 'Año'], right_on=['Código Departamento', 'Año'], how='left')
dfModelo = dfModelo.drop(columns=['Código Departamento'])
dfModelo = dfModelo.rename(columns={'Rendimiento (ton_ha)': 'rendimiento'})
dfModelo = pd.merge(dfModelo, evalsIndCosecA, left_on=['CodDep', 'Año'], right_on=['Código Departamento', 'Año'], how='left')
dfModelo = dfModelo.dropna()
dfModelo = dfModelo[['Año', 'CodDep', 'valorobservado_temp', 'valorobservado_vel', 'Índice de Cosecha', 'rendimiento']]
dfModelo

In [ ]:
# @title Entrenamiento
resultados_por_departamento = {}

departamentos = dfModelo['CodDep'].unique()

for dep in departamentos:
    df_dep = dfModelo[dfModelo['CodDep'] == dep]

    X = df_dep[['Año', 'valorobservado_temp', 'valorobservado_vel', 'Índice de Cosecha']]
    y = df_dep['rendimiento']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    riesgo_agricola = (y_pred.max() - y_pred) / (y_pred.max() - y_pred.min())

    resultados_dep = X_test.copy()
    resultados_dep['prediccion_rendimiento'] = y_pred
    resultados_dep['riesgo_agricola'] = riesgo_agricola
    resultados_dep['CodDep'] = dep

    resultados_por_departamento[dep] = {
        'mse': mse,
        'r2': r2,
        'resultados': resultados_dep
    }

resultados_finales = pd.concat([resultados_por_departamento[dep]['resultados'] for dep in departamentos])

riesgo_por_departamento = resultados_finales.groupby(['Año', 'CodDep'])[['riesgo_agricola']].mean().reset_index()

print("Riesgo agrícola por departamento y año:")
display(riesgo_por_departamento)

### Precipitaciones

### Velocidad del Viento

## Mapas

In [ ]:
# @title Mapa Folium
with open('/content/drive/MyDrive/DANE/colombia.geojson', 'r') as f:
    colombia_geojson = json.load(f)

m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
# m = folium.Map(location=[4.5709, -74.2973], zoom_start=6, width='50%', height='70%')

# area_dict = evalsSemb.set_index('Código Departamento')['Área Sembrada (ha)'].to_dict()

folium.Choropleth(
    geo_data=colombia_geojson,
    data=evalsSemb,
    columns=['Código Departamento', 'Área Sembrada (ha)'],
    key_on='feature.properties.DPTO_CCDGO',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Área Sembrada (ha)'
).add_to(m)

m

### Área Sembrada

In [ ]:
# @title Promedio de Área Sembrada por Departamento

for feature in colombia_geojson['features']:
    feature['id'] = feature['properties']['DPTO']

fig = px.choropleth(
    evalsSemb,
    geojson=colombia_geojson,
    locations='Código Departamento',
    featureidkey='properties.DPTO',
    color='Área Sembrada (ha)',
    color_continuous_scale="YlGnBu",
    title="Área Sembrada (ha) por Departamento"
)

fig.update_geos(
    visible=False,
    fitbounds="locations",
    projection_type="mercator"
)

fig.update_layout(
    width=700,
    height=700
)

fig.show()

In [ ]:
# @title Promedio de Área Sembrada por Departamento (anual)

min_value = evalsSembA['Área Sembrada (ha)'].min()
max_value = evalsSembA['Área Sembrada (ha)'].max()

fig = px.choropleth(
    evalsSembA,
    geojson=colombia_geojson,
    locations='Código Departamento',
    featureidkey='properties.DPTO',
    color='Área Sembrada (ha)',
    color_continuous_scale="YlGnBu",
    range_color=(min_value, max_value),
    animation_frame='Año',
    title="Área Sembrada (ha) por Departamento (Evolución Anual)"
)

fig.update_geos(
    visible=False,
    fitbounds="locations",
    projection_type="mercator"
)

fig.update_layout(
    width=700,
    height=700,
    sliders=[{
        "steps": [{"label": str(year), "method": "animate", "args": [[str(year)], {"mode": "immediate"}]} for year in evalsSembA['Año'].unique()],
        "currentvalue": {"prefix": "Año: "}
    }]
)

fig.show()

### Rendimiento

In [ ]:
# @title Promedio del Rendimiento por Departamento

for feature in colombia_geojson['features']:
    feature['id'] = feature['properties']['DPTO']

fig = px.choropleth(
    evalsRend,
    geojson=colombia_geojson,
    locations='Código Departamento',
    featureidkey='properties.DPTO',
    color='Rendimiento (ton_ha)',
    color_continuous_scale="YlGnBu",
    title="Rendimiento (ton_ha) por Departamento"
)

fig.update_geos(
    visible=False,
    fitbounds="locations",
    projection_type="mercator"
)

fig.update_layout(
    width=700,
    height=700
)

fig.show()

In [ ]:
# @title Promedio del Rendimiento por Departamento (anual)

min_value = evalsRendA['Rendimiento (ton_ha)'].min()
max_value = evalsRendA['Rendimiento (ton_ha)'].max()

fig = px.choropleth(
    evalsRendA,
    geojson=colombia_geojson,
    locations='Código Departamento',
    featureidkey='properties.DPTO',
    color='Rendimiento (ton_ha)',
    color_continuous_scale="YlGnBu",
    range_color=(min_value, max_value),
    animation_frame='Año',
    title="Rendimiento (ton_ha) por Departamento (Evolución Anual)"
)

fig.update_geos(
    visible=False,
    fitbounds="locations",
    projection_type="mercator"
)

fig.update_layout(
    width=700,
    height=700,
    sliders=[{
        "steps": [{"label": str(year), "method": "animate", "args": [[str(year)], {"mode": "immediate"}]} for year in evalsRendA['Año'].unique()],
        "currentvalue": {"prefix": "Año: "}
    }]
)

fig.show()

### Mapa de Riesgo

In [ ]:
# @title Mapa de Riesgo Agrícola Folium

riesgoAgricola = riesgo_por_departamento.groupby('CodDep')['riesgo_agricola'].mean().reset_index()

m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
# m = folium.Map(location=[4.5709, -74.2973], zoom_start=6, width='50%', height='70%')

# area_dict = evalsSemb.set_index('CodDep')['Área Sembrada (ha)'].to_dict()

folium.Choropleth(
    geo_data=colombia_geojson,
    data=riesgoAgricola,
    columns=['CodDep', 'riesgo_agricola'],
    key_on='feature.properties.DPTO',
    fill_color='YlOrRd',
    fill_opacity=0.9,
    line_opacity=0.2,
    legend_name='Índice de Riesgo Agrícola',
    highlight=True,
    line_color='black',
    nan_fill_opacity=0.075,
    nan_fill_color='green',
).add_to(m)

m